In [1]:
# IMPORTS from tvm, different required packages and the profiling infrastructure

from numpy.core.numeric import full
import tvm
from tvm.contrib import utils, graph_executor as runtime
from tvm.contrib.debugger import debug_executor as graph_runtime
from tvm.relay.op.nn.nn import dense, dilate, conv2d
#####
import numpy as np
import pynvml as nv
from func_timeout import func_timeout
import time
import psutil
import json
#####
from components import description_vector as dv
from components import serializer
from components import profiling

In [2]:
repeat = 1024

In [3]:
# helpful to suppress output of debug runtime run function
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [4]:
# defining important variables for the profiling system

target = "cuda"
target_class = "cuda"

device = "980ti"
dev_idx = 1
dev = tvm.device(str("cuda"), dev_idx)
time_min_res = 0.2


state_path = "./states"
state_file = "state"

layer_name = "conv2d"

metrics = profiling.get_metrics(target, device, backend="nvml", dev_idx=dev_idx)
print(metrics)

['nvml:::NVIDIA_GeForce_GTX_980_Ti:device_1:power']


In [5]:
nv.nvmlInit()
handle = nv.nvmlDeviceGetHandleByIndex(0)

#metrics.append("nvml:::NVIDIA_GeForce_GTX_980_Ti:device_1:pstate") #unable to read using TVM PAPI Profiler due to limitation to integer data
metrics.append("nvml:::NVIDIA_GeForce_GTX_980_Ti:device_"+str(dev_idx)+":gpu_utilization")
metrics.append("nvml:::NVIDIA_GeForce_GTX_980_Ti:device_"+str(dev_idx)+":memory_utilization")
metrics.append("nvml:::NVIDIA_GeForce_GTX_980_Ti:device_"+str(dev_idx)+":graphics_clock")
metrics.append("nvml:::NVIDIA_GeForce_GTX_980_Ti:device_"+str(dev_idx)+":sm_clock")
metrics.append("nvml:::NVIDIA_GeForce_GTX_980_Ti:device_"+str(dev_idx)+":memory_clock")
metrics.append("nvml:::NVIDIA_GeForce_GTX_980_Ti:device_"+str(dev_idx)+":allocated_memory")

In [7]:
import json
with open('conv_layer_config.json') as file:
    configs = json.load(file)
    print(type(configs))
print(len(configs))

<class 'dict'>
2627


In [ ]:
measurements = {}
data_layout = "NCHW"
kernel_layout = "OIHW"

for name, config in configs.items():
    print(config["output shape"])
    
    for batch_size in [1, 8, 16, 32, 64, 128]:
        file = "dataset/"+target_class+"_"+device+"_"+name.replace(":", "-")+"_"+str(batch_size)+".json"
        if os.path.exists(file):
            print("already profiled")
            break
        required = int(np.prod(config["input shape"][1::])*batch_size)
        inp_shape = (
            int(batch_size),
            int(config["input shape"][3]),
            int(config["input shape"][2]),
            int(config["input shape"][1]),
        )
        
        rand_data = np.random.rand(int(np.ceil(required/repeat)))
        inp_data = np.repeat(rand_data, repeat)[:required].reshape(inp_shape).astype("float32")
        
        weight_shape = (
            int(config["output shape"][3]),
            int(config["input shape"][3]),
            int(config["kernel"][0]),
            int(config["kernel"][1])
        )
        
        required = int(config["output shape"][3]) * int(config["input shape"][3] / config["groups"]) * int(config["kernel"][0]) * int(config["kernel"][1])
        rand_data = np.random.rand(int(np.ceil(required/repeat)))
        weight_data = np.repeat(rand_data, repeat)[:required].reshape(weight_shape).astype("float32")

        x = tvm.relay.var("data", tvm.relay.TensorType(inp_shape), dtype="float32")
        y = tvm.relay.Constant(tvm.nd.array(weight_data))
        
        try:
            expr = conv2d(
                data = x,
                weight= y,
                strides=int(config["strides"][0]),
                padding=0,
                dilation=int(config["dilation"][0]),
                groups=int(config["groups"]),
                channels=int(config["output shape"][3]),
                kernel_size=int(config["kernel"][0]),
                data_layout=data_layout,
                kernel_layout=kernel_layout,
            )

            mod = tvm.ir.IRModule.from_expr(expr)
            params = {}
            with tvm.transform.PassContext(opt_level=3):
                compiled_graph_lib = tvm.relay.build_module.build(mod, target_class, params=params)
        except:
            print("compile failed")
            break
            
        try:
            debug_g_mod = graph_runtime.GraphModuleDebug(
                compiled_graph_lib["debug_create"]("default", dev),
                [dev],
                compiled_graph_lib.get_graph_json(),
                "."
            )
        except:
            print("runtime construction failed")
            break
        
        try:
            t_start  = time.monotonic()
            times = debug_g_mod.run_individual(10, 3, 1000)
            t_end = time.monotonic()
        except:
            print("execution time measurement failed")
            break
            
        for idx, node in enumerate(debug_g_mod.debug_datum._nodes_list):
            if layer_name in node["op"]:
                layer_time = float(times[idx])*1000
                actual_layer_name = node["op"]
                
        runs = int(max(1, np.ceil(time_min_res / (layer_time/1000))))

        # determine the noise
        iterations = 10
        powers = []
        gpu_utils = []
        mem_utils = []
        gpu_clocks = []
        sm_clocks = []
        mem_clocks = []
        alloc_memory = []
        profile_times = []
                
        try:
            t_burn_in = 5
            t_start = time.monotonic()
            t_end = t_start + t_burn_in
            while time.monotonic() < t_end:
                # run debug runtime without profiling as burn in
                with suppress_stdout():
                    test_data = debug_g_mod.profile(collectors=[], data=tvm.nd.array(inp_data.astype("float32")), runs=runs)

            p_start = time.monotonic()
            for r in range(0, iterations):        
                # reload the Metric Collector due to issues with the PAPI backend
                data_collector = tvm.runtime.profiling.PAPIMetricCollector({dev: metrics}, component="nvml")    

                # run debug runtime with time measurements only
                #with suppress_stdout():
                test_data = debug_g_mod.profile(collectors=[data_collector], data=tvm.nd.array(inp_data.astype("float32")), runs=runs)
                pstate = nv.nvmlDeviceGetPowerState(handle)
                #print("\r",(r+1),"PState:", pstate, end="")

                # extract measurement of current run
                powers.append(test_data.calls[0][metrics[0]].value)
                gpu_utils.append(test_data.calls[0][metrics[1]].value)
                mem_utils.append(test_data.calls[0][metrics[2]].value)
                gpu_clocks.append(test_data.calls[0][metrics[3]].value)
                sm_clocks.append(test_data.calls[0][metrics[4]].value)
                mem_clocks.append(test_data.calls[0][metrics[5]].value)
                alloc_memory.append(test_data.calls[0][metrics[6]].value)
                profile_times.append(test_data.calls[0]["Duration (us)"].microseconds/1000000/runs) # in seconds
                #time.sleep(1)

            p_delta = time.monotonic() - p_start
            avg_power = np.mean(powers)/1000
            max_power = np.max(powers)/1000
            min_power = np.min(powers)/1000
            std_power = np.std(powers)/1000
            #calculate Z-Score
            z_scores = ((np.array(powers)/1000) - avg_power)/std_power
            cleaned_powers = []
            threshold = 0.25
            while len(cleaned_powers) < 3:
                cleaned_powers = []
                threshold += 0.05
                for idx, score in enumerate(z_scores):
                    if abs(score) < threshold:
                        cleaned_powers.append(powers[idx]/1000)

            layer_power = np.median(cleaned_powers)
            layer_memory = np.median(alloc_memory)/(1024**3)

            #print()
            measurements[name+"_"+str(batch_size)] = (layer_time, layer_power, layer_memory)
            print(name, batch_size, (layer_time, layer_power, layer_memory))
        except:
            print("failed power measurement")
            break
            
        config["data layout"] = data_layout
        config["kernel layout"] = kernel_layout
        config["batch_size"] = batch_size

        json_text = json.dumps(config)
        with open(file, "w") as file:
            file.write(json_text)
            #print()
        
            #measurements[state] = (-1, -1, -1)
            

[None, 149, 149, 32]


[14:54:12] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 0
[14:54:12] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 45.0134 us/iter
[14:54:13] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 1
[14:54:13] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 45.2078 us/iter
[14:54:14] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 2
[14:54:14] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 45.1694 us/iter


xception:block1_conv1 1 (0.0451694, 172.872, 0.10174560546875)


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
[14:54:25] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 0
[14:54:25] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 2151.07 us/iter
[14:54:26] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 1
[14:54:26] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 2151.53 us/iter
[14:54:28] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 2
[14:54:28] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 2151.55 us/iter


xception:block1_conv1 8 (2.15155, 183.934, 0.12725830078125)


...

vgg16:block4_conv2 64 (143.80100000000002, 226.043, 0.28399658203125)


[15:18:58] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 0
[15:18:58] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 287220 us/iter
[15:19:01] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 1
[15:19:01] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 287232 us/iter
[15:19:03] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 2
[15:19:03] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 287176 us/iter


vgg16:block4_conv2 128 (287.176, 226.697, 0.46221923828125)
[None, 28, 28, 512]


[15:19:25] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 0
[15:19:25] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 2395.14 us/iter
[15:19:26] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 1
[15:19:26] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 2395.21 us/iter
[15:19:28] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:103: Iteration: 2
[15:19:28] /home/max/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:108: Op #0 tvmgen_default_fused_nn_conv2d: 2396.61 us/iter


failed power measurement
[None, 14, 14, 512]


In [ ]:
measurements